In [51]:
import os

import pandas as pd
import Python as py

In [52]:
root = os.getcwd()
df_scheduled = pd.read_csv(root + "/data/scheduled_loan_repayments.csv")
df_actual = pd.read_csv(root + "/data/actual_loan_repayments.csv")
df_balances = py.calculate_df_balances(df_scheduled, df_actual)

In [23]:
## Question 1
# print(df_balances[df_balances['ActualRepayment'] < df_balances['ScheduledRepayment']])
filtered_df = df_balances[df_balances['ActualRepayment'] < df_balances['ScheduledRepayment']]
count_distinct_loans_type1default = filtered_df['LoanID'].nunique()
count_distinct_loans = df_balances['LoanID'].nunique()
default_rate_percent = 100 * count_distinct_loans_type1default / count_distinct_loans
print(default_rate_percent)
print(py.question_1(df_balances))

15.0
15.0


In [57]:
# What timeframes are we dealing with?
print(max(df_balances["Month"])) # max month = 12 
print(min(df_balances["Month"])) # min month = 12
## First need to calculcate how many payments are expected
# print(df_balances[df_balances["ActualRepayment"] == 0].head())

## Check all scheduled payments for each loan 
repayments_per_loan = df_scheduled.groupby("LoanID").size()

## Unpaid payments are defined a payment being 0
unpaid_payments = df_balances[df_balances["ActualRepayment"] == 0].groupby("LoanID").size()

## Create a new DF that is on a loan-grain
loans_df = pd.DataFrame({
    'TotalExpectedRepayments': repayments_per_loan,
    'UnpaidPayments': unpaid_payments
})

## Update NaN values to 0 for calculcation purposes:
loans_df['UnpaidPayments'] = loans_df['UnpaidPayments'].fillna(0).astype(int)

## Calculate percentage of payments in a loan that were not repaid:
loans_df["PercentUnpaidPayments"] = 100 * loans_df["UnpaidPayments"] / loans_df["TotalExpectedRepayments"]

## Calculate what % of loans are in type 2 default:
    default_rate_percent = 100 * len(loans_df[loans_df["PercentUnpaidPayments"] > 15]) / len(loans_df)
print(default_rate_percent)

IndentationError: unexpected indent (1650773188.py, line 26)

In [19]:
## Question 3
## SMM is calculated as: (Unscheduled Principal)/(Start of Month Loan Balance)

## We want to calculate unscheduled prinicpal first. This is the amount of the loan payment that goes towards the principal
## The scheduled prinicpal would be scheduled repayment less the interest payment
df_balances["UnscheduledPrincipal"] = df_balances["ScheduledRepayment"] - df_balances["InterestPayment"]

## SMM is calculated as: (Unscheduled Principal)/(Start of Month Loan Balance)
df_balances["SMM"] = df_balances["UnscheduledPrincipal"] / df_balances["LoanBalanceStart"]

## SMM_mean is calculated as (∏(1+SMM))^(1/12) - 1
SMM_mean = (df_balances["SMM"] ** (1/12)).prod() -1
print(SMM_mean)

## CPR is calcualted as: 1 - (1- SMM_mean)^12
cpr = 1 - (1 - SMM_mean) ** 12
print(cpr)

-1.0
-4095.0


In [123]:
## Question 4
## probability_of_default is taken from question 1 or question 2 answer
## Assume a recovery rate of 80%


## Need to calculate the total loan balance at the end of year 1
## To do this, sum all the loan balances at the end of month 12:
yearend_loan_balance = df_balances.loc[df_balances["Month"] == 12, "LoanBalanceEnd"].sum()

## Recovery rate is given to be 0.8
recovery_rate = 0.8

## Probabiltiy of default is taken from question 2
probability_of_default = 0.1

## Predicted total loss: probability_of_default * total_loan_balance * (1 - recovery_rate)
total_loss = probability_of_default * yearend_loan_balance * (1 - recovery_rate)
print(py.question_3(df_balances))

NameError: name 'cpr_percent' is not defined